# Introductory demo of Core Imaging Library (CIL) 

### First 3D cone-beam data of walnut, then 3D parallel-beam data of steel wire

First import all modules we will need:

In [ ]:
import numpy as np
import os

from cil.io import TXRMDataReader
from cil.processors import TransmissionAbsorptionConverter
from cil.plugins.astra import FBP
from cil.utilities.display import show2D

from cil.processors import CentreOfRotationCorrector, Slicer
from cil.io import TIFFWriter

## Walnut, cone-beam case
Load the 3D cone-beam projection data of a walnut:

In [ ]:
filename = "/media/newhd/shared/Data/zeiss/walnut/valnut/valnut_2014-03-21_643_28/tomo-A/valnut_tomo-A.txrm"
data = TXRMDataReader(file_name=filename).read()

We can call print for the data to get some basic information:

In [ ]:
print(data)

Note how labels refer to the different dimensions. We can use labels to extract and display 2D slices of data, such as a single projection:

In [ ]:
show2D(data, slice_list=('angle',800))

From the background value of 1.0 we infer that the data is transmission data (it is known to be already centered and flat field corrected) so we just need to convert to absorption/apply the negative logarithm, which can be done using a CIL processor, which will handle small/large outliers:

In [ ]:
data = TransmissionAbsorptionConverter()(data)

We again take a look at a slice of the data, now a vertical one to see the central slice sinogram after negative logarithm:

In [ ]:
show2D(data, slice_list=('vertical',512))

CIL supports different back-ends for which data order conventions may differ. Here we use the FBP algorithm from the ASTRA Toolbox, which requires us to permute the data array into the right order:

In [ ]:
data.reorder(order='astra')

The data is now ready for reconstruction. To set up the FBP algorithm we must specify the size/geometry of the reconstruction volume. Here we use the default one:

In [ ]:
ig = data.geometry.get_ImageGeometry()

We can then create the FBP algorithm (really FDK since 3D cone-beam) from ASTRA running on the GPU and reconstruct the data:

In [ ]:
fbp =  FBP(ig, data.geometry, "gpu")
recon = fbp(data)

We show the central (vertical) slice:

In [ ]:
show2D(recon, slice_list=('vertical',512), fix_range=(-0.01,0.06))

And a horizontal (horizontal_x) slice as well:

In [ ]:
show2D(recon, slice_list=('horizontal_x',512), fix_range=(-0.01,0.06))

We can save the reconstructed volume to disk for example as a stack of TIFFs:

In [ ]:
TIFFWriter(data=recon, file_name="/media/newhd/shared/Data/out").write()

## Steel-wire, parallel-beam case

The steel-wire data set is included in CIL as a demonstration data set that can be loaded by:

In [ ]:
from cil.utilities.dataexample import SYNCHROTRON_PARALLEL_BEAM_DATA
data_sync = SYNCHROTRON_PARALLEL_BEAM_DATA.get()

We take a look at the data with "print":

In [ ]:
print(data_sync)

And show the first projection:

In [ ]:
show2D(data_sync.get_slice(angle=0), origin='upper-left')

White background indicates transmission data, but value is not 1.0 as expected for air. Not sure why that is, but we can correct this by normalising with the background value estimated over some region, here we extract a single line and compute its mean:

In [ ]:
scale = data_sync.get_slice(vertical=20).mean()
print(scale)
data_sync = data_sync/scale

As before we convert to absorption (take negative log)

In [ ]:
data_sync = TransmissionAbsorptionConverter()(data_sync)

This data set is NOT corrected for centre-of-rotation offset. We can do this using the CIL Processor CentreOfRotationCorrector, here with a simple cross-correlation method on the central slice:

In [ ]:
data_sync = CentreOfRotationCorrector.xcorrelation(slice_index='centre')(data_sync)

The data contains a redundant projection at 180 degrees, which can be discarded by keeping only the 90 angles. At the same time air on both sides can be cropped off by keeping only horizontal pixels from 20 to 140 out of 160. This is done using the Slicer Processor and the trimmed data is printed, showing the horizontal dimension now reduced to 120:

In [ ]:
data90 = Slicer(roi={'angle':(0,90), 
                     'horizontal':(20,140,1)})(data_sync)

print(data90)

We show one of the cropped, centered and negative-log transformed projections:

In [ ]:
show2D(data90.subset(angle=0), origin='upper-left')

And the sinogram for a selected slice:

In [ ]:
show2D(data90.subset(vertical=103), origin='upper-left')

Again we must reorder before we can use the ASTRA back-end:

In [ ]:
data90.reorder(order='astra')

Then we set up and run the FBP algorithm from the acquisition and default image geometry. For a 3D data set like this, a 2D FBP is run slice by slice:

In [ ]:
ag = data90.geometry
ig = ag.get_ImageGeometry()

recon90 = FBP(ig, ag, device='gpu')(data90)

We show a horizontal and a vertical slice of the reconstructed volume:

In [ ]:
show2D(recon90.get_slice(horizontal_x=44), cmap='inferno', fix_range=(-0.01,0.11), origin='upper-left')
show2D(recon90.get_slice(vertical=103)   , cmap='inferno', fix_range=(-0.01,0.11), origin='upper-left')

It is illustrative to look quickly at reconstruction from reduced data, such as fewer projections. Here we simulate having only 15 projections by throwing away all but every sixth projection using a Slicer Processor. We print and note that there are only 15 angles now:

In [ ]:
data15 = Slicer(roi={'angle': (0,90,6)})(data90)
print(data15)

As before we set up and run FBP reconstruction:

In [ ]:
recon15 = FBP(ig, data15.geometry, device='gpu')(data15)

We show the same horizontal and vertical slices now showing big artifacts from having fewer projections:

In [ ]:
show2D(recon15.get_slice(horizontal_x=44), cmap='inferno', fix_range=(-0.01,0.11), origin='upper-left')
show2D(recon15.get_slice(vertical=103)   , cmap='inferno', fix_range=(-0.01,0.11), origin='upper-left')